# AMEX

## Libraries and functions

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

from scipy.stats import zscore
import gc
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.set_option('display.max_columns', None)
import warnings 
warnings.filterwarnings("ignore")
#Generic libraryies
import random
import math
import itertools

#Time and monitoring libraries
import time
from tqdm import tqdm
import pyprind

#Visual libraries
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)
import matplotlib.dates as mdates
import seaborn as sns

#Processing data
from sklearn.preprocessing import RobustScaler , MinMaxScaler , power_transform
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.impute import MissingIndicator

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import sys
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/amex-parquet/test_data.parquet
/kaggle/input/amex-parquet/train_data.parquet


In [2]:
def scaler(df, features):
    df[features] = power_transform(df[features].values)
    return df
    
def missingSW(df, features, drop = False):
    imp = MissingIndicator()
    names = [ f"{j}M" for j in features ]
    df[names] = imp.fit_transform(df[features])
    for j in names:
        df[j] = df[j].apply(lambda x : 1 if x == True else 0)
    if drop:
        df = df.drop(features, axis = 1)
    return df

def imputerLR(df, features, seed = 95):
    imp_mean = IterativeImputer(random_state=seed)
    names = [f"{x}MLr" for x in features]
    imp_mean.fit(df)
    df[names] = imp_mean.transform(df[features])
    df = df.drop(features, axis = 1)
    return df

## Load data and group features

In [3]:
categorical = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']
target = "target"
dates = "S_2"
seed = 95

# We will use parquet format of the dataset created by @odins0n for some data exploration. Parquet is fater and save the dtypes of each columns when we read and write.
%time train = pd.read_parquet("/kaggle/input/amex-parquet/train_data.parquet")
print()
train.info()

CPU times: user 10.4 s, sys: 13.5 s, total: 23.9 s
Wall time: 38.3 s

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5531451 entries, 0 to 5531450
Columns: 191 entries, customer_ID to target
dtypes: float32(185), int64(2), object(4)
memory usage: 4.1+ GB


In [4]:
aux = pd.DataFrame(train.isna().mean().sort_values(ascending = False), columns = ["nans"] ).reset_index()

hugeMissing = aux[aux.nans > 0.95]['index'].values
alotMissing = aux[ (aux.nans > 0.25) & (aux.nans <= 0.95) ]['index'].values
lessMissing = aux[aux.nans <= 0.25]['index'].values

del aux
gc.collect()

0

## Featuring engeneering

* Huge missing values -> More than 95% drop column
* A lot of missing values -> Sw columns with 1 or 0 (more than 25% nans values and less than 95%) then drop column
* Scaler -> Power transformer for the numerical features. 
* Missing values -> Less than (25%) impute with LR, because LR is robuster agains overfitting.
* Group customers and extract the most information possible 
    1. Take duration max - min dates
    2. Min, Max, mean, std, most frequent, etc. of the features
    3. Rolling values, taking -7 days, -1 month, -3 month, 6-month
    4. 

In [5]:
#1 
train.drop(hugeMissing, axis = 1, inplace = True)
gc.collect()

21

In [6]:
# 2
train = missingSW(train, alotMissing , True)
gc.collect()

21

In [7]:
train.head()

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_44,B_4,D_45,B_5,R_2,D_46,D_47,D_48,B_6,B_7,B_8,D_51,B_9,R_3,D_52,P_3,B_10,S_5,B_11,S_6,D_54,R_4,S_7,B_12,S_8,D_55,B_13,R_5,D_58,B_14,D_59,D_60,D_61,B_15,S_11,D_62,D_63,D_64,D_65,B_16,B_18,B_19,B_20,D_68,S_12,R_6,S_13,B_21,D_69,B_22,D_70,D_71,D_72,S_15,B_23,P_4,D_74,D_75,B_24,R_7,B_25,B_26,D_78,D_79,R_8,S_16,D_80,R_10,R_11,B_27,D_81,S_17,R_12,B_28,R_13,D_83,R_14,R_15,D_84,R_16,B_30,S_18,D_86,R_17,R_18,B_31,S_19,R_19,B_32,S_20,R_20,R_21,B_33,D_89,R_22,R_23,D_91,D_92,D_93,D_94,R_24,R_25,D_96,S_22,S_23,S_24,S_25,S_26,D_102,D_103,D_104,D_107,B_36,B_37,R_27,B_38,D_109,D_112,B_40,D_113,D_114,D_115,D_116,D_117,D_118,D_119,D_120,D_121,D_122,D_123,D_124,D_125,D_126,D_127,D_128,D_129,B_41,D_130,D_131,D_133,R_28,D_139,D_140,D_141,D_143,D_144,D_145,target,R_9M,B_29M,D_106M,D_132M,D_49M,R_26M,D_76M,D_66M,D_42M,D_142M,D_53M,D_82M,D_50M,B_17M,D_105M,D_56M,S_9M,D_77M,D_43M,S_27M
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-03-09,0.938469,0.001733,0.008724,1.006838,0.009228,0.124035,0.008771,0.004709,0.000630,0.080986,0.708906,0.170600,0.006204,0.358587,0.525351,0.255736,0.063902,0.059416,0.006466,1.335856,0.008207,0.001423,0.207334,0.736463,0.096219,0.023381,0.002768,0.008322,1.001519,0.008298,0.161345,0.148266,0.922998,0.354596,0.118075,0.001882,0.158612,0.018385,0.063646,0.199617,0.308233,0.016361,0.401619,0.091071,CR,O,0.007126,0.007665,0.652984,0.008520,0.004730,6.0,0.272008,0.008363,0.515222,0.002644,0.009013,0.004808,0.008342,0.119403,0.004802,0.108271,0.050882,0.007554,0.080422,0.069067,0.004327,0.007562,0.007729,0.000272,0.001576,0.004239,0.001434,0.002271,0.004061,0.007121,0.002456,0.002310,0.003532,0.008033,1.009825,0.084683,0.003820,0.007043,0.000438,0.006452,0.000830,0.005055,0.0,0.005720,0.007084,0.000198,0.008907,1,0.002537,0.005177,0.006626,0.009705,0.007782,0.002450,1.001101,0.002665,0.007479,0.006893,1.503673,1.006133,0.003569,0.008871,0.003950,0.003647,0.004950,0.894090,0.135561,0.911191,0.974539,0.001243,0.766688,1.008691,1.004587,0.670041,0.009968,0.004572,1.008949,2.0,0.004326,1.007336,0.210060,0.007871,1.0,0.238250,0.0,4.0,0.232120,0.236266,0.0,0.702280,0.434345,0.003057,0.686516,0.008740,1.0,1.003319,1.007819,1.000080,0.006805,0.002052,0.005972,0.004345,0.001535,0.002427,0.003706,0.003818,0.000569,0.000610,0.002674,0,1,1,1,1,1,1,1,1,1,1,1,0,0,1,0,0,0,1,1,0
1,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-04-07,0.936665,0.005775,0.004923,1.000653,0.006151,0.126750,0.000798,0.002714,0.002526,0.069419,0.712795,0.113239,0.006206,0.353630,0.521311,0.223329,0.065261,0.057744,0.001614,1.339794,0.008373,0.001984,0.202778,0.720886,0.099804,0.030599,0.002749,0.002482,1.009033,0.005136,0.140951,0.143530,0.919414,0.326757,0.118737,0.001610,0.148459,0.013035,0.065501,0.151387,0.265026,0.017688,0.406326,0.086805,CR,O,0.002413,0.007148,0.647093,0.002238,0.003879,6.0,0.188970,0.004030,0.509048,0.004193,0.007842,0.001283,0.006524,0.140611,0.000094,0.101018,0.040469,0.004832,0.081413,0.074166,0.004203,0.005304,0.001864,0.000979,0.009896,0.007597,0.000509,0.009810,0.000127,0.005966,0.000395,0.001327,0.007773,0.000760,1.009461,0.081843,0.000347,0.007789,0.004311,0.002332,0.009469,0.003753,0.0,0.007584,0.006677,0.001142,0.005907,1,0.008427,0.008979,0.001854,0.009924,0.005987,0.002247,1.006779,0.002508,0.006827,0.002837,1.503577,1.005791,0.000571,0.000391,0.008351,0.008850,0.003180,0.902135,0.136333,0.919876,0.975625,0.004561,0.786007,1.000084,1.004118,0.668647,0.003921,0.004654,1.003205,2.0,0.008707,1.007653,0.184093,0.003444,1.0,0.247217,0.0,4.0,0.243532,0.241885,0.0,0.707017,0.430501,0.001306,0.686414,0.000755,1.0,1.008394,1.004333,1.008344,0.004407,0.001034,0.004838,0.007495,0.004931,0.003954,0.003167,0.005032,0.009576,0.005492,0.009217,0,1,1,1,1,1,1,1,1,1,1,1,0,0,1,0,0,0,1,1,0
2,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-05-28,0.954180,0.091505,0.021655,1.009672,0.006815,0.123977,0.007598,0.009423,0.007605,0.068839,0.720884,0.060492,0.003259,0.334650,0.5